In [2]:
cd /Users/alessandropreviero/Desktop/vaccine-allocation/src

/Users/alessandropreviero/Desktop/vaccine-allocation/src


In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.graph_objects as go
import plotly.express as px
import us
import os

In [4]:
import main
from models.prescriptive_delphi_model import *
from pipeline.data_loading import *
from pipeline.data_processing import *
from pipeline.constants import *
from pipeline.scenario import *

In [7]:
scenario_params_grid = [
        dict(
            **dates,
            vaccine_effectiveness=vaccine_effectiveness,
            daily_vaccine_budget=daily_vaccine_budget,
            min_allocation_factor=min_allocation_factor
        )
        for dates in DATES_GRID
        for vaccine_effectiveness in VACCINE_EFFECTIVENESS_GRID
        for daily_vaccine_budget in DAILY_VACCINE_BUDGET_GRID
        for min_allocation_factor in MIN_ALLOCATION_FACTOR_GRID
    ]

scenario_params = scenario_params_grid[0]

start_date = scenario_params["start_date"]
end_date = scenario_params["end_date"]
mortality_rate_path = f"{MORTALITY_RATES_PATH}2021.npy"
reload_mortality_rate = os.path.isfile(mortality_rate_path)

model = Scenario(**scenario_params).load_model(
    mortality_rate_path=mortality_rate_path
)

fake_vaccines = np.zeros(shape=(51, 6, 91))
solution = model.simulate(vaccinated=fake_vaccines)

In [8]:
deaths = solution.deceased[:,:,-1].sum(axis=1)
deaths.sum()

70902.81254085636

In [16]:
deaths = solution.deceased[:,:,-1].sum(axis=1)
states = pd.read_csv('/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/county_pop_clean.csv')
states_names = states['state'].unique()
deaths_no_vaccines = pd.DataFrame(deaths, columns=['deaths'])
deaths_no_vaccines['state'] = [us.states.lookup(x).name for x in states_names]
deaths_no_vaccines.to_csv("deaths_no_vaccines.csv")